<a href="https://colab.research.google.com/github/minsaee/github_workspace/blob/master/650_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/ai_chat_python'

Mounted at /content/drive
/content/drive/MyDrive/ai_chat_python


In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 24.0 MB/s eta 0:00:00


### 단어 임베딩
 - 단어 임베딩은 말뭉치에서 각각의 단어를 벡터로 변환하는 기법이다.

1. 원-핫 인코딩
 - 메모리 차지가 크다. -> 손실이 크다.
 - 단어들의 관계를 나타낼 수 없다. -> 의미를 해석할 수 없다.
 - 값의 타입을 1과0으로 나타낸다.

In [3]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)

# 원-핫 인코딩
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

['오늘', '날씨', '구름']
{'오늘': 0, '날씨': 1, '구름': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


2 희소표현과 분산표현
 - 원-핫 인코딩은 표현하고자 하는 단어의 인덱스 요소만 1이고, 나머지 요소는모두 0으로 표현되는 희소 벡터(희소행렬)이다.  
 - 단어가 희소 벡터로 표현되는 방식은 희소표현(Sparse Representation)이라 부른다.
 - 희소표현은 각각의 차원이 독립적인 정보를 지니고 있어사람이 이해하기에는 직관적인 장점이 있다.
 - 단어 사전의 크기가 커질수록 메모리 낭비와 계산 복잡도가 커지는 단점이 있다.
 - 단어 간의 연관성이 전혀 없어 의미를 담을 수 없다.
 - 자연어처리를 잘하기 위해서는 기본 토큰이 되는 단어의 의미와 주변단어간의 관계를 단어 임베딩에 표현되어야한다.
 - 희소표현으로는 해결하기 힘들기에, 각 단어 간의 유사성을잘 표현하면서도 벡터 공간을 절약할 수 있는 방법을 분산표현(Distributed Representation)이라한다.
 - 분산표현은 한 단어의 정보가 특정 차원에 표현되지않고 여러 차원에 분산되어 표현되어 붙여진 이름이다.
 - 하나의 차원에 다양한 정보를 가지고 있다.
 - RGB모델은 3차원 형태의 벡터로, 분산표현 방식이다.
 - 분산표현은 우리가 원하는 차원에데이터를 최대한 밀집시킬 수 있어 밀집표현(Dense Representation)이라고 부른다.
 - 밀집표현으로 만들어진 벡터를 밀집벡터(Dense Vector)라한다.

1_3 Word2Vec
- Word2Vec은 분산 표현 형태의 단어 임베딩 모델이다.
- Word2Vec은 2013년 구글에서 발표했으며 가장 많이 사용하고 있는 단어 임베딩 모델이다.
- Word2Vec 모델은 CBOW(continuous bag-of-words)와 skip-gram 두가지 모델이다.
- CBOW 모델 맥락(context word)이라 표현되는 주변 단어들을 이용해 타깃단어를 예측하는 신경망 모델이다. 신경망의 입력을 주변 단어들로 구성하고 출력을 타깃 단어로 설정해 학습된 가중치 데이터를 임베딩 벡터로 활용한다.
- skip-gram모델은 CBOW 모델과 반대로 하나의 타깃 단어를 이용해 주변 단어들을 예측하는 신경망 모델이다. 입출력이 CBOW모델과 반대로 되어 있기 때문에 skip-gram모델이 CBOW 모델에 비해 예측해야 하는 맥락이 많아진다. 따라서, 단어 분산 표현력이 우수해 CBOW 모델에 비해 임베딩 품질이 우수하다. 반면 CBOW모델은 타깃 단어의 손실만 계산하면 되기 때문에 학습 속도가 빠른 장점이 있다.
- CBOW 모델에서는 타깃 단어를 예측하기 위해 앞뒤 단어를 확인할 수 있다. 이때 앞뒤로 몇개의 단어까지 확인할지 결정할 수 있는데 이 범위를 윈도우(window)라고한다. 예를 들어 윈도우 크기를 2로 한 경우 타깃 단어의 앞뒤 단어 2개까지 모델 학습을 위해 사용된다.
- Word2Vec의 단어 임베딩은 해당 단어를 밀집 벡터로 표현하며 학습을 통해 의미상 비슷한 단어들을 비슷한 벡터 공간에 위치시킨다. 또한 벡터 특성상 의미에 따라 방향을 갖게 된다.
- 임베딩된 벡터들 간 연산이 가능하기 때문에 단어 간 관계를 계산할 수 있다. '왕'과'여왕'의 방향 차이만큼 '남자'와'여자'의 방향 차이가 생긴다.

In [5]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time


# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header 제거
    return data


# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./dataset/ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료:  2.4444162845611572
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  181.45132112503052


In [11]:
# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1)
print('3) word2vec 모델 학습 완료:', time.time()-start)
# vector_size : 단어 임베딩 벡터의 차원(크기), window: 주변 단어 윈도우 크기
# hs : 1(모델 학습에 softmax 사용), 0(negative 옵션 값이 0이 아닐 때 음수 샘플링으로 사용)
# min_count : 단어 최소 빈도 수 제한 (설정된 min_count 빈도 수 이하의 단어들은 학습하지 않음)
# sg : 0(CBOW 모델), 1(skip-gram 모델)

# 모델 저장하기.
print('4) 학습된 모델 저장 시작')
model.save('./dataset/nvmc.model')
print('4) 학습된 모델 저장 완료', time.time()-start)

# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료: 1375.1302483081818
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료 1375.2439498901367
corpus_count :  200000
corpus_total_words :  1076896


In [19]:
print(model.wv.index_to_key) # list
print(model.wv.key_to_index) # dict

['영화', '것', '점', '연기', '수', '거', '평점', '최고', '생각', '때', '스토리', '드라마', '사람', '말', '감동', '!!', '배우', '내용', '감독', '안', '시간', '중', '재미', '듯', '쓰레기', '작품', '번', '사랑', '건', '이', '마지막', '이건', '개', '액션', '기대', '만', '!!!', '장면', '완전', '처음', '정도', '편', '씨', '내', '느낌', '이야기', '최악', '돈', '끝', '이해', '명작', '분', '한국', '주인공', '여자', '연출', '이영화', '때문', '년', '기억', '일', '진짜', '다', '한', '줄', '데', '마음', '인생', '애', '결말', '현실', '짜증', '소재', '굿', '작', '수준', '공포', '반전', '전개', '몰입', '필요', '일본', '매력', '음악', '넘', '가슴', '눈물', '속', '원작', '지', '남자', '급', '인간', '전', '추천', '뿐', '지금', '남', '노', '자체', '이상', '실망', '아', '게', '눈', '캐릭터', '개봉', '여운', '정말', '볼', '코미디', '모습', '연기력', '글', '가족', '후', '중간', '오', '대', '공감', '표현', '대박', '부분', '제목', '대사', '기분', '티', '진심', '스릴러', '작가', '시리즈', '영상', '억지', '애니', '삶', '요즘', '물', '극장', '명', '욕', '긴장감', '추', '이랑', '강', '부족', '졸', '존', '개인', '시작', '영화로', '허', '자', '발', '알바', '임', '노래', '막장', '의미', '나름', '이유', '제일', '당시', '요', '오랜만', '친구', '시대', '미', '식', '점수', '미국', '팬', '절대', '라고', '리', '감

In [15]:
from gensim.models import Word2Vec

# 모델 로딩
model = Word2Vec.load('./dataset/nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print(len( model.wv['사랑'])) # model size = 200
print('사랑 : ', model.wv['사랑'])

# 단어 유사도 계산
# model.wv.similarity( ) : 두 단어 간의 유사도를 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

corpus_total_words :  1076896
200
사랑 :  [-0.03015657 -0.67968774  0.06031771  0.18364605 -0.23555763 -0.045322
 -0.04417454  0.38817874 -0.08433481  0.36160684 -0.16966239 -0.246878
  0.05365005 -0.3176684  -0.24495913  0.4251952  -0.10615171 -0.45046064
 -0.38542655 -0.33203143  0.2725931   0.25715506  0.09148119  0.15919836
 -0.14748509  0.02043674  0.05485079 -0.05439676 -0.33626893 -0.30064595
  0.16794544  0.22477008  0.17368303  0.11977711 -0.13640772  0.355082
  0.03084124 -0.11458718 -0.09451043 -0.15458232 -0.21456774  0.16962549
 -0.124111   -0.34458086  0.53517264  0.44191036  0.0567467  -0.29399586
  0.49684933  0.15298767  0.02280407 -0.14868623 -0.09055723 -0.01395723
  0.0385549  -0.27076983  0.04778107 -0.3199359  -0.1550075   0.18723881
  0.01643841  0.02722017 -0.45213354 -0.00701573 -0.09642347 -0.1761201
 -0.42402267  0.04715587 -0.05710649  0.669859    0.14578222  0.08035856
  0.32911408  0.17289993  0.36467743 -0.12759653  0.28080505 -0.29282194
 -0.2607904  -0.35

In [18]:
# 가장 유사한 단어 추출
# model.wv.most_similar( ) : 인자로 사용한 단어와 가장 유사한 단어를 리스트로 반환한다.
# 즉, 벡터 공간에서 가장 가까운 거리에 있는 단어들을 반환한다. topn인자는 반환되는 유사한 단어 수를 의미하며,
# 예제에서는 5개까지 유사한 단어를 반환한다.
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))
print(model.wv.most_similar(["안성기","최고"], topn=5))
print(model.wv.most_similar(["안성기","최고"], negative=['재발견'], topn=5))

[('정재영', 0.951984167098999), ('설경구', 0.9504806399345398), ('최강희', 0.9493293166160583), ('박신양', 0.944958508014679), ('김하늘', 0.9422689080238342)]
[('미션', 0.8090018630027771), ('엑스맨', 0.8089837431907654), ('다이하드', 0.805766761302948), ('포터', 0.8055742979049683), ('미이라', 0.8037148714065552)]
[('이범수', 0.8815878629684448), ('공리', 0.8787795901298523), ('김래원', 0.8764088749885559), ('재발견', 0.868858277797699), ('설경구', 0.865419328212738)]
[('최강', 0.7555714845657349), ('단언', 0.7258887887001038), ('넘버원', 0.7189612984657288), ('최악', 0.70552659034729), ('명불허전', 0.7050729990005493)]
